In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import warnings

warnings.filterwarnings('ignore')

train_path = 'C:/Users/f3107/Desktop/hy_data/hy_round1_train_20200102'
test_path = 'C:/Users/f3107/Desktop/hy_data/hy_round1_testA_20200102'

In [2]:
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)
print(len(train_files), len(test_files))

7000 2000


In [3]:
train_files[:3]

['0.csv', '1.csv', '10.csv']

In [4]:
test_files[:3]

['7000.csv', '7001.csv', '7002.csv']

In [5]:
df = pd.read_csv(f'{train_path}/6966.csv')

In [6]:
df.head()

,渔船ID,x,y,速度,方向,time,type
0,6966,6.265902e+06,5.279254e+06,0.11,306,1106 23:58:16,围网
1,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:48:21,围网
2,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:38:19,围网
3,6966,6.265902e+06,5.279254e+06,0.00,0,1106 23:28:36,围网
4,6966,6.265902e+06,5.279254e+06,0.32,130,1106 23:08:17,围网


In [7]:
df['type'].unique()

array(['围网'], dtype=object)

In [8]:
df.shape

(389, 7)

In [9]:
ret = []
for file in tqdm(train_files):
    df = pd.read_csv(f'{train_path}/{file}')
    ret.append(df)
df = pd.concat(ret)
df.columns = ['ship','x','y','v','d','time','type']

100%|█████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:37<00:00, 185.09it/s]


In [10]:
df.to_hdf('C:/Users/f3107/Desktop/hy_data/train.h5', 'df', mode='w')

In [11]:
ret = []
for file in tqdm(test_files):
    df = pd.read_csv(f'{test_path}/{file}')
    ret.append(df)
df = pd.concat(ret)
df.columns = ['ship','x','y','v','d','time']

100%|█████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:11<00:00, 177.93it/s]


In [12]:
df.to_hdf('C:/Users/f3107/Desktop/hy_data/test.h5', 'df', mode='w')

In [13]:
df.shape

(782378, 6)

In [14]:
df.head()

,ship,x,y,v,d,time
0,7000,7.118845e+06,5.918277e+06,0.11,0,1103 11:54:32
1,7000,7.118940e+06,5.918285e+06,0.32,346,1103 11:44:32
2,7000,7.118948e+06,5.918174e+06,0.11,0,1103 11:34:43
3,7000,7.118948e+06,5.918174e+06,0.11,71,1103 11:14:30
4,7000,7.118948e+06,5.918174e+06,0.11,30,1103 11:04:46


In [21]:
train01 = pd.read_csv(train_path +'/201.csv')
train01.head()

,渔船ID,x,y,速度,方向,time,type
0,201,6.246324e+06,5.241263e+06,0.22,294,1106 23:56:50,拖网
1,201,6.246324e+06,5.241263e+06,0.11,0,1106 23:46:54,拖网
2,201,6.246324e+06,5.241263e+06,0.11,0,1106 23:36:53,拖网
3,201,6.246324e+06,5.241263e+06,0.00,0,1106 23:26:46,拖网
4,201,6.246324e+06,5.241263e+06,0.32,186,1106 23:06:51,拖网


In [23]:
test01 = pd.read_csv(test_path +'/7000.csv')
test01.head()

,渔船ID,x,y,速度,方向,time
0,7000,7.118845e+06,5.918277e+06,0.11,0,1103 11:54:32
1,7000,7.118940e+06,5.918285e+06,0.32,346,1103 11:44:32
2,7000,7.118948e+06,5.918174e+06,0.11,0,1103 11:34:43
3,7000,7.118948e+06,5.918174e+06,0.11,71,1103 11:14:30
4,7000,7.118948e+06,5.918174e+06,0.11,30,1103 11:04:46
